<a href="https://colab.research.google.com/github/AndresPautrat/Concurrente/blob/main/go_forks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt install golang-go

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  golang-1.10-go golang-1.10-race-detector-runtime golang-1.10-src
  golang-race-detector-runtime golang-src
Suggested packages:
  bzr mercurial subversion
The following NEW packages will be installed:
  golang-1.10-go golang-1.10-race-detector-runtime golang-1.10-src golang-go
  golang-race-detector-runtime golang-src
0 upgraded, 6 newly installed, 0 to remove and 21 not upgraded.
Need to get 40.3 MB of archives.
After this operation, 225 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 golang-1.10-src amd64 1.10.4-2ubuntu1~18.04.1 [11.1 MB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 golang-1.10-go amd64 1.10.4-2ubuntu1~18.04.1 [28.6 MB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/main amd64 golang-1.10-race-detector-runtime amd64 0.0+svn285455-0ubuntu1 [478

In [ ]:
%%writefile main.go
package main
import (
    "fmt"
)

func proc(inC <-chan float64, outC chan<- float64){
    min := float64(1<<62)
    for num :=range inC{
        if num<min{
            outC<-min
            min=num
        }else{
            outC<-num
        }
    }
    fmt.Printf("%.0f",min)
    close(outC)
}

func main(){
    nums := []float64{6.,1.,8.,10.,5.,9.,3.,7.,2.,4.}
    ch := make([]chan float64,len(nums)+1)
    ch[0]=make(chan float64)
    for i :=range nums{
        ch[i+1]=make(chan float64)
        go proc(ch[i],ch[i+1])
    }
    go func(){
        for _,num:=range nums{
            ch[0]<-num
        }
        close(ch[0])
    }()
    for range ch[len(nums)]{
        
    }
}

Overwriting main.go


In [ ]:
!go run main.go

12345678910

In [ ]:
%%writefile 1.go
package main
import (
    "fmt"
)

func main(){
    ach:=make(chan string, 1)
    ach<-"Hola"
    fmt.Println(<-ach)
}

Overwriting 1.go


In [ ]:
!go run 1.go

Hola


In [ ]:
%%writefile dp.go
package main
import (
    "fmt"
)

func philosopher(leftFork chan bool, rigthFork chan bool){
    for {
      fmt.Println("Thinking")
      <-leftFork
      <-rigthFork
      fmt.Println("Eating")
      leftFork<-true
      rigthFork<-true
    }
}
func fork(chFork chan bool){
    for{
        chFork<-true
        <-chFork
    }
}

func main(){
    forks:=make([]chan bool,5)
    for i:= range forks{
        forks[i]=make(chan bool,1)
    }
    go philosopher(forks[0],forks[1])
    go fork(forks[0])
    go philosopher(forks[1],forks[2])
    go fork(forks[1])
    go philosopher(forks[2],forks[3])
    go fork(forks[2])
    go philosopher(forks[3],forks[4])
    go fork(forks[3])
    go philosopher(forks[4],forks[0])
    fork(forks[4])
}



Overwriting dp.go


In [ ]:
!go run dp.go

In [ ]:
%%writefile dp2.go
package main
import (
    "fmt"
)

func p(id int,ch chan string){
    for i := 0; i<10;i++{
        ch<-fmt.Sprintf("%d hola %d", id, i)
    }
}

func main(){
    ch1 := make(chan string)
    ch2 := make(chan string)

    go p(1,ch1)
    go p(2,ch2)

    for i := 0; i < 10; i++{
        select{
            case msg1:=<-ch1:
              fmt.Printf("%s %s\n",msg1,<-ch2)
            case msg2 := <-ch2:
              fmt.Printf("%s %s\n",msg2, <-ch1)
        }
    }
}

Overwriting dp2.go


In [ ]:
!go run dp2.go

1 hola 0 2 hola 0
1 hola 1 2 hola 1
2 hola 2 1 hola 2
1 hola 3 2 hola 3
2 hola 4 1 hola 4
2 hola 5 1 hola 5
2 hola 6 1 hola 6
2 hola 7 1 hola 7
1 hola 8 2 hola 8
2 hola 9 1 hola 9
